# Text Classification

This is our `hello world` example 

In [1]:
# a helper functions and imports
from IPython.display import display
def highlight_col(x, df):
    #set by condition
    mask =  df['label'] == 'pos'
    mask2 = df['label'] == 'neg'
    x = pd.DataFrame('', index=df.index, columns=df.columns)
    x.loc[mask] = 'background-color: #e6ffe6'
    x.loc[mask2] = 'background-color: #ffe6e6'
    return x    

## Load the data

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

train_pos = pd.read_csv("../data/arabic-sentiment-twitter-corpus/train_Arabic_tweets_positive_20190413.tsv",
                        sep='\t', header=None)
train_neg = pd.read_csv("../data/arabic-sentiment-twitter-corpus/train_Arabic_tweets_negative_20190413.tsv",
                        sep='\t', header=None)
train_data = pd.concat([train_pos, train_neg], ignore_index=True)
train_data.columns = ['label', 'tweet']
del train_pos, train_neg

pd.set_option('display.max_colwidth', 100000)
df_tmp = train_data.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
13571,pos,كلابي وينكم مالكم حس 👣
11456,pos,ابتسامات الورد والفل لصباحك 🌷
33633,neg,انباء تؤكد غياب منصور الحربي عن لقاء النصر اليوم السبت بسبب الاصابة .. ☹ - البديل العمار الذي قدم مستوى اكثر من جي…
33296,neg,باقي انا اشد حيلي بالايديت ويطلع حلو 💔
18006,pos,بمناسبة فوز الهلال .. 💙 سحب على آيفون XR📱 رتويت وتابع - السحب بعد ساعة موثق بالفديو 💪


In [17]:
test_pos = pd.read_csv("../data/arabic-sentiment-twitter-corpus/test_Arabic_tweets_positive_20190413.tsv",
                        sep='\t', header=None)
test_neg = pd.read_csv("../data/arabic-sentiment-twitter-corpus/test_Arabic_tweets_negative_20190413.tsv",
                        sep='\t', header=None)
test_data = pd.concat([test_pos, test_neg], ignore_index=True)
test_data.columns = ['label', 'tweet']
del test_pos, test_neg

df_tmp = test_data.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
50,pos,هذا الفيديو فيه معنى المثل اللي يقول ( ضربني وبكى وسبقني واشتكى ) 😂 عموما ودي داكوستا يجلده صدق عشان يعرف يمثل مره…
10449,neg,اللهم املأ قبر ميتنا بالرضا والنور والفسحة والسرور ياعزيز ياغفور .. #اللهم_ارحم_الوالد_اخوي 💔
2576,pos,كل شيء احبه فيه ، متغير وكل شيء ماحبه على حاله :)
2632,pos,خارج النص | - فرحة مشجع عماني في نهائي كأس جلالة السلطان قابوس 😂
5820,neg,ღ♬ . . أسلى مع طيفك الين إنتعشتك عشان مااحس بغيابك على طول ياجعلها في ذمتك ماوحشتك 💔 #تصميمي…


## Baseline model (using pipeline)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(train_data.tweet, train_data.label);

/home/ali/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Let's test this basic model

In [50]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, test_data.tweet, test_data.label)

              precision    recall  f1-score   support

         neg       0.77      0.82      0.80      5768
         pos       0.81      0.76      0.78      5752

    accuracy                           0.79     11520
   macro avg       0.79      0.79      0.79     11520
weighted avg       0.79      0.79      0.79     11520

accuracy: 0.792


## let's take a look inside the model

In [54]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

Weight?,Feature
+2.779,الإخونج
+2.414,وصباحك
+2.216,هالسنه
+2.093,ابريل
+2.092,السحب
+2.086,الزرقاء
+2.075,برونو
+2.029,اللوك
+1.887,الطيب
+1.880,حكمة


## Try our model on some tweets

In [58]:
for _, row in test_data.sample(5).iterrows():
    print(f"true label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vec,))
    print("--"*50)

true label: pos


----------------------------------------------------------------------------------------------------
true label: pos


----------------------------------------------------------------------------------------------------
true label: pos


----------------------------------------------------------------------------------------------------
true label: neg


----------------------------------------------------------------------------------------------------
true label: neg


----------------------------------------------------------------------------------------------------


## Try Tfidf with some processing

In [61]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)
clf = LinearSVC()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(train_data.tweet, train_data.label)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('linearsvc', LinearSVC())])

In [62]:
print_report(pipe_tfidf, test_data.tweet, test_data.label)

              precision    recall  f1-score   support

         neg       0.85      0.82      0.84      5768
         pos       0.83      0.86      0.84      5752

    accuracy                           0.84     11520
   macro avg       0.84      0.84      0.84     11520
weighted avg       0.84      0.84      0.84     11520

accuracy: 0.838


In [63]:
eli5.show_weights(clf, vec=vec, top=20)

In [64]:
for _, row in test_data.sample(5).iterrows():
    print(f"true label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vec,))
    print("--"*50)

true label: pos


----------------------------------------------------------------------------------------------------
true label: pos


----------------------------------------------------------------------------------------------------
true label: pos


----------------------------------------------------------------------------------------------------
true label: neg


----------------------------------------------------------------------------------------------------
true label: neg


----------------------------------------------------------------------------------------------------


# Feel free to play with notebook explore different models with different datasets